In which I attempt to use the Camera controller to play nicely with the camera despite being agnostic about the units.

In general, if there is a fatal bluetooth error, disconnect and then reconnect to the swarm.



In [2]:
import sphero

# Controller Libraries
import time
import os
import json
from pprint import pprint
import math

In [3]:
# Tracking code
import cv2

# Modify from Simon Tordensky's Base 
from tracker.trackingfilter import FilterSpheroBlueCover, FilterGlow
from tracker.traceable import TraceableObject
from tracker.trackerbase import ColorTracker

from util import Vector2D

In [4]:
from SpheroController.tracablesphero import TraceableSphero

In [5]:
# Library written to help with tracking
import SpheroTeam
from SpheroTeam.teamutil import readJsonFile, normalize_angle, angle_between_points

## Part 1: Connecting to the Sphero "Team"

Requirements for configuration files:



### Populating `roster.json`
- Use the Sphero iPhone app to get each robot's bluetooth name and MAC address.
    - Bluetooth Name is the name of the robot when connected to your phone (i.e. Sphero-RGB)
    - MAC address can be determined by going to the options menu of the robot.
- You could also use the SpheroManager's discovery options to get your robots, but I don't recommend it if you're going to be using the same robots again and again.

### Populating `config.json`

Pick a standard set of colors to use with your fleet, specified with RGB notation


In [6]:
# Config management
PROJ_ROOT = os.getcwd()
CONFIG = os.path.join(PROJ_ROOT, "config.json")

In [7]:
# Class for managing multiple spheros
manager = sphero.SpheroManager()

# Initialize Sphero manager with addresses of local spheros
manager= SpheroTeam.initialize.load_sphero_roster(manager, CONFIG)

In [8]:
# Verify that proper robots have been added
for name, device in manager._spheros.iteritems():
    print "{}: {}".format(name, device.bt_addr)

Sphero-RPB: 68:86:E7:05:12:26
Sphero-YPR: 68:86:E7:09:4E:74
Sphero-GRY: 68:86:E7:09:A9:28
Sphero-YYP: 68:86:E7:05:0C:4B
Sphero-RWR: 68:86:E7:09:A2:FE
Sphero-ORG: 68:86:E7:09:A6:FE
Sphero-RYR: 68:86:E7:05:19:AD


In [9]:
# Manually pick out the robots to include by specifying their abbreviation.
# example: bots = [devices[4], devices[1]]
activeBots = ['YPR']

In [10]:
# Connect to these robots
bots = SpheroTeam.initialize.connect_sphero_team(manager, activeBots)

Sphero-YPR Try #0
IOError Occurred. Trying to connect again.
Sphero-YPR Try #1
Connected!


In [11]:
# All robots start with lights off to save power
for bot in bots:
    bot.set_rgb(0,0,0)

In [12]:
# Check team power state
SpheroTeam.print_team_status(bots)

Sphero-YPR:Battery OK | 8.01 V


### Part 2: Integrating the Camera


In [13]:
#Track Camera Size: 
tracker = ColorTracker()

# Check dimensions of your field. We're using the Logitech C920
tracker.image_size

(640, 480)

In [14]:
# Use blue because it's the most reliable color filter
traceable_blue = TraceableObject("BLUE")
traceable_blue.filter = FilterSpheroBlueCover()

In [15]:
# For Debugging
def display_current_view(tracker):
    image = tracker.get_video_frame()
    cv2.imshow("img", image)
    cv2.waitKey(0)
    
    cv2.destroyAllWindows()
    
def display_current_video(tracker):
    """
        Displays current video feed, press 'q' to escape
    """
    cam = tracker.cam
    while(True):
        # Capture frame-by-frame
        ret, frame = cam.read()

        # Display the resulting frame
        cv2.imshow('frame', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    cv2.destroyAllWindows()

    
# For Navigation
def get_bot_position(bot, traceable_object, tracker, samples=3, debug=False):  
    xSamples = []
    ySamples = []

#     traceable_object.screen_size = tracker.image_size
     
    for sample in range(samples):
        image = tracker.get_video_frame()
        timestamp = time.time()
        
        if sample > 0: # ignore the first sample
            x, y = tracker._find_traceable_in_image(image, traceable_object) # side effect: adds mask to tracker  
            if x:
                xSamples.append(x)
            if y:
                ySamples.append(y)

            traceable_object.add_tracking(Vector2D(x, y), timestamp)

    if debug:      
        display_current_view(tracker)
    print "{} | {} ".format(xSamples, ySamples)
        
#     return traceable_object.pos.x, traceable_object.pos.y
    return (sum(xSamples) / len(xSamples)), (sum(ySamples) / len(ySamples))


def calibrate_bot_direction(bot, traceable_object_list, traceable_color, tracker, debug=False):
    """
        Routine for making all robots line up in the same direction
        
    """
#     traceable_object = traceable_object_list[0]
    TIMEOUT = 1500  # in MILLISECONDS
    
    
    bot.set_rgb(traceable_color[0], traceable_color[1], traceable_color[2])
    bot.set_motion_timeout(TIMEOUT)
    startX, startY =  get_bot_position(bot, traceable_object_list[0], tracker)

#     traceable_object_list = tracker.track_object(traceable_object_list)
#     startX, startY = traceable_object_list[0].pos.x, traceable_object_list[0].pos.y
    cv2.waitKey(250)  # not sure how long this has to be
    
    
    if (startX is None) or (startY is None):
        print("Error: Robot not in view")
        return -1
    
    print "Start ({},{})".format( startX, startY)

    bot.roll(60, 0)
    time.sleep(TIMEOUT / 1000) 

    endX, endY       =  get_bot_position(bot, traceable_object_list[0], tracker)
#     traceable_object_list = tracker.track_object(traceable_object_list)
#     endX, endY = traceable_object_list[0].pos.x, traceable_object_list[0].pos.y
      
    print "End   ({},{})".format(   endX, endY)
    offset = normalize_angle(angle_between_points(startX, startY, endX, endY))
    

    print "Angle {}".format(offset) 
#     bot.roll(50, offset)
#     time.sleep(1.5)
#     bot.set_heading(offset)
    
    
    # Turn lights off 
    
#     bot.set_rgb(0, 0, 0)
    
    return offset

# Video display
def display_tracking_window(traceable_object_list, exitKey = "q"):
    """
        Given a list of iterables, track all objects in that list.
        Press exitKey to quit the window
    """
    
    while(True):
        #     tracklist = tracker.track_objects(tracklist)

        tracker.track_objects(tracklist)
        cv2.waitKey(1)

        if cv2.waitKey(1) & 0xFF == ord(exitKey):
            break 
    cv2.destroyAllWindows()


In [16]:
# Print what the camera is currently seeing as a debugging step
# display_current_view(tracker)

In [17]:
# Check which bot is which
SpheroTeam.highlight_team(bots)

In [18]:
# Check which bots are ready
SpheroTeam.print_team_status(bots)


Sphero-YPR:Battery OK | 8.01 V


In [19]:
tracklist = [traceable_blue]

In [34]:
offsets = []
for bot in bots:
    bot.set_rgb(0, 0, 255)
    time.sleep(2)
    
    offsets.append(calibrate_bot_direction(bot, tracklist, [0, 0, 255], tracker, True))
    
    bot.set_rgb(0,0,0)
    
    proceed = raw_input("Next Robot")

[449, 449] | [283, 283] 
Start (449,283)
[421, 416] | [294, 296] 
End   (418,295)
Angle 158.838740183
Next Robotd


In [37]:
# Check if robot rolls horizontally across the the room

SpheroTeam.roll_sphero_team_synchronized(bots, 60, 180, offsets, 1000)


Dispatch Time 0.0339999198914


In [38]:
# Check video
display_current_view(tracker)



### Scrap Section

Code below is unstructured, and some are outputs that were used to develop the above sections.



In [32]:
print traceable_blue.pos

(x: 281, y:156, deg:29.0372889653)


In [70]:
ts.device.roll(40, 180)

In [28]:
print traceable_blue.pos

(x: 444, y:201, deg:24.3563912528)


In [118]:
get_bot_position(bot, traceable_blue, tracker, samples=4, debug=False)

[183, 279, 279, 279] | [247, 241, 241, 241] 


(279, 241)

In [63]:
ts = TraceableSphero(bot)

In [53]:
print traceable_blue.pos

(x: 340, y:254, deg:36.7618944194)


In [19]:
ts.tracking_samples = []

image = tracker.get_video_frame()
x, y = tracker._find_traceable_in_image(image, traceable_blue)
ts.add_tracking(Vector2D(x, y), time.time())

print x
print y

143
371


In [56]:
ts.calibrate_direction()

starts calibration
True


In [36]:
get_bot_position(bots[0], traceable_blue, tracker, debug=False)

[454, 454, 454, 454] | [202, 202, 202, 202] 


(454, 202)

[341] | [254] 
[340] | [254] 
Start (341,254)
End   (340,254)
Angle 180.0


180.0

In [85]:
bot.roll(50,270)

In [32]:
bot.roll(50, 135)

time.sleep(1.5)
bots[0].set_heading(0)

In [109]:
tracker._draw_masks()
cv2.waitKey(1)

Debugging Section